# Lesson 2.1 - Experiment Tracking with W&B / MLflow

***Key Concepts:*** *Experiment Trackers, Weights & Biases, MLflow*

When training your own models, you can easily get overwhelmed by the large number of different experiments you conduct and you might find yourself losing track of how different hyperparameters affected your model performance, or what exact configuration produced the best model you had trained. That is why experiment tracking tools like Tensorboard, Weights & Biases, or MLflow are often one of the first touchpoints you will have with MLOps as you progress through your ML journey.

In this lesson, we will learn about two of the most popular tools in this space: Weights & Biases and MLflow. Let's integrate them into our pipelines and see them in action!

To get started, run the following commands to install both tools with their respective dependencies.

In [ ]:
!zenml integration install mlflow -f
!zenml integration install wandb -f

Next, let's import our pipeline definition and some of the pipeline steps that we built in the previous lessons:

In [ ]:
from src.steps.evaluator import evaluator
from src.steps.importer import importer
from src.pipelines.digits_pipeline import digits_pipeline

## MLFlow

The only thing we need to change now to start using MLflow is the `svc_trainer` step. To do so, we define a new step `svc_trainer_mlflow` in which we use MLflow's `mlflow.sklearn.autolog()` feature to automatically log all relevant attributes of our model to MLflow. By adding an `@enable_mlflow` decorator on top of the function, ZenML then automatically initializes MLflow and takes care of the rest for us.

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
import mlflow
from zenml.integrations.mlflow.mlflow_step_decorator import enable_mlflow
from zenml.steps import step, BaseStepConfig


def build_svc_mlflow_pipeline(gamma=1e-3):
    @enable_mlflow  # setup MLflow
    @step(
        enable_cache=False
    )  # disable caching so we log **every** run to MLflow
    def svc_trainer_mlflow(
        X_train: np.ndarray,
        y_train: np.ndarray,
    ) -> ClassifierMixin:
        """Train a sklearn SVC classifier and log to MLflow."""
        mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
        model = SVC(gamma=gamma)
        model.fit(X_train, y_train)
        return model

    return digits_pipeline(
        importer=importer(),
        trainer=svc_trainer_mlflow(),
        evaluator=evaluator(),
    )

Now, let's do the same for our decision tree trainer step:

In [ ]:
from sklearn.tree import DecisionTreeClassifier


def build_tree_mlflow_pipeline():
    @enable_mlflow  # setup MLflow
    @step(
        enable_cache=False
    )  # disable caching so we log **every** run to MLflow
    def tree_trainer_with_mlflow(
        X_train: np.ndarray,
        y_train: np.ndarray,
    ) -> ClassifierMixin:
        """Train a sklearn decision tree classifier and log to MLflow."""
        mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        return model

    return digits_pipeline(
        importer=importer(),
        trainer=tree_trainer_with_mlflow(),
        evaluator=evaluator(),
    )

Finally, to run our MLflow pipelines with ZenML, we first need to add MLflow into our MLOps stack.
To do so, we first register a new experiment tracker with ZenML and then add it into our current stack.
We can use `zenml stack describe` to show an overview of our currently active MLOps stack.

In [ ]:
# Register the MLflow experiment tracker
!zenml experiment-tracker register mlflow_tracker --type=mlflow

# Add the MLflow experiment tracker into our default stack
!zenml stack update default -e mlflow_tracker

# See an overview of your current MLOps stack
!zenml stack describe

Now we're all setup so we can simply call `pipeline.run()` for all of our pipelines, which will now automatially log all of our pipeline runs to MLflow. Let's try it out and do a few pipeline runs with different hyperparameters:

In [ ]:
digits_pipeline_svc_mlflow.run()

In [ ]:
for gamma in (1e-4, 1e-3, 1e-2, 1e-1):
    config = SVCTrainerConfig(gamma=gamma)
    digits_pipeline_svc_mlflow.with_config(config).run()

In [ ]:
digits_pipeline_tree_mlflow.run()

To compare all of our runs within the MLflow UI, run the following cell, then open http://127.0.0.1:4997/ in your browser.

In [ ]:
# This will start a serving process for mlflow
#  - if you want to continue in the notebook you need to manually
#  interrupt the kernel
from zenml.integrations.mlflow.mlflow_utils import get_tracking_uri

!mlflow ui --backend-store-uri="{get_tracking_uri()}" --port=4997

## Weights & Biases

To get this example running, you need to set up a Weights & Biases account. You can do this for free [here](https://wandb.ai/login?signup=true).

In [ ]:
# Register the W&B experiment tracker
!zenml experiment-tracker register wandb_tracker --type=wandb --api_key=<WANDB_API_KEY> --entity=<WANDB_ENTITY> --project_name=<WANDB_PROJECT_NAME>

# Create a new stack with W&B experiment tracker in it
!zenml stack register wandb_stack -m default -a default -o default -e wandb_tracker

# Set it as the active stack
!zenml stack set wandb_stack

# See an overview of your current MLOps stack
!zenml stack describe

Note that despite wandb being used in different steps within a pipeline, ZenML handles initializing wandb and ensures the experiment name is the same as the pipeline name, and the experiment run is the same name as the pipeline run name. This establishes a lineage between pipelines in ZenML and experiments in wandb.

In [ ]:
import numpy as np
import wandb
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin
from zenml.integrations.wandb.wandb_step_decorator import enable_wandb
from zenml.steps import step


@enable_wandb(wandb.Settings(magic=True))
@step
@step(enable_cache=False)
def svc_trainer_wandb(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train another simple sklearn classifier for the digits dataset."""
    wandb.log()  # TODO
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    return model